# Custom LLM

This notebook goes over how to create a custom LLM wrapper, in case you want to use your own LLM or a different wrapper than one that is supported in LangChain.

Wrapping your LLM with the standard `LLM` interface allow you to use your LLM in existing LangChain programs with minimal code modifications!

As an bonus, your LLM will automatically become a LangChain `Runnable` and will benefit from some optimizations out of the box (e.g., batch via a threadpool), async support, the `astream_events` API, etc.

## Implementation

There are only two required things that a custom LLM needs to implement:

- A `_call` method that takes in a string, some optional stop words, and returns a string.
- A `_llm_type` property that returns a string. Used for logging purposes only. 

There is a second optional thing it can implement:

- An `_identifying_params` property that is used to help with printing of this class. Should return a dictionary.

Let's implement a very simple custom LLM that just returns the first n characters of the input.

In [74]:
from typing import Any, List, Mapping, Optional, Dict

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

In [75]:
class CustomLLM(LLM):
    """A custom chat model that echoes the first `n` characters of the input.

    When contributing an implementation to LangChain, carefully document
    the model including the initialization parameters, include
    an example of how to initialize the model and include any relevant
    links to the underlying models documentation or API.

    Example:

        .. code-block:: python

            model = CustomChatModel(n=2)
            result = model.invoke([HumanMessage(content="hello")])
            result = model.batch([[HumanMessage(content="hello")],
                                 [HumanMessage(content="world")]])
    """

    n: int
    """The number of characters from the last message of the prompt to be echoed."""


    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Run the LLM on the given input.

        Override this method to implement the LLM logic.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of the stop substrings.
                If stop tokens are not supported consider raising NotImplementedError.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            The model output as a string. Actual completions SHOULD NOT include the prompt.
        """
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return prompt[: self.n]

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters.

        IMPORTANT:
            The LangChain callback system and caching absractions use identifying parameters
            to track the configuration with which the LLM call was made.
            If identifying parameters are not properly specified, then important tracing
            information will be missing in callbacks (e.g., users may not be able to set up
            analytics for the model) and caching of model calls will be incorrect.
        """
        return {
            "n": self.n,
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }
    
    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"
    

### Let's test it 🧪

This LLM will implement the standard `Runnable` interface of LangChain which many of the LangChain abstractions support!

In [67]:
llm = CustomLLM(n=3)

In [68]:
print(llm)

CustomLLM
Params: {'n': 3, 'model_name': 'CustomChatModel'}


In [69]:
llm.invoke("This is a foobar thing")

'Thi'

We can also print the LLM and see its custom print.

In [70]:
await llm.ainvoke('world')

'wor'

In [71]:
llm.batch(['woof woof woof', 'meow meow meow'])

['woo', 'meo']

In [72]:
await llm.abatch(['woof woof woof', 'meow meow meow'])

['woo', 'meo']

In [73]:
 llm.abatch(['woof woof woof', 'meow meow meow'])

<coroutine object BaseLLM.abatch at 0x7805d28a0760>

It'll benfit from other standard runnable methods